## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anchorage,US,2020-08-25 05:56:53,61.22,-149.90,57.20,87,75,4.70,light rain
1,1,Okhtyrka,UA,2020-08-25 06:00:02,50.31,34.90,71.08,44,79,8.21,broken clouds
2,2,Okakarara,NaN,2020-08-25 06:00:03,-20.58,17.43,57.52,36,0,5.14,clear sky
3,3,Hilo,US,2020-08-25 05:53:44,19.73,-155.09,80.60,69,75,4.70,broken clouds
4,4,Kushima,JP,2020-08-25 06:00:04,31.46,131.23,91.40,70,20,14.99,shower rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hilo,US,2020-08-25 05:53:44,19.73,-155.09,80.60,69,75,4.70,broken clouds
8,8,Tasgaon,IN,2020-08-25 06:00:05,17.03,74.60,81.57,68,62,13.04,broken clouds
9,9,Vaini,TO,2020-08-25 05:53:24,-21.20,-175.20,77.00,88,20,4.70,few clouds
10,10,Saint-Philippe,RE,2020-08-25 05:54:05,-21.36,55.77,75.20,64,39,6.93,scattered clouds
24,24,Ixtapa,MX,2020-08-25 05:53:41,20.70,-105.20,82.40,78,90,0.87,overcast clouds
25,25,Kapaa,US,2020-08-25 05:53:30,22.08,-159.32,80.60,74,40,11.41,scattered clouds
29,29,Adrar,MR,2020-08-25 05:53:56,20.50,-10.07,88.70,40,0,18.52,clear sky
41,41,Kirakira,SB,2020-08-25 06:00:12,-10.45,161.92,83.62,72,12,7.18,light rain
46,46,Sao Filipe,CV,2020-08-25 05:54:22,14.90,-24.50,77.32,80,31,1.72,scattered clouds
56,56,Hamilton,US,2020-08-25 05:52:43,39.18,-84.53,78.01,78,1,5.82,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                226
City                   226
Country                226
Date                   226
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,80.60,broken clouds,19.73,-155.09,
8,Tasgaon,IN,81.57,broken clouds,17.03,74.60,
9,Vaini,TO,77.00,few clouds,-21.20,-175.20,
10,Saint-Philippe,RE,75.20,scattered clouds,-21.36,55.77,
24,Ixtapa,MX,82.40,overcast clouds,20.70,-105.20,
25,Kapaa,US,80.60,scattered clouds,22.08,-159.32,
29,Adrar,MR,88.70,clear sky,20.50,-10.07,
41,Kirakira,SB,83.62,light rain,-10.45,161.92,
46,Sao Filipe,CV,77.32,scattered clouds,14.90,-24.50,
56,Hamilton,US,78.01,clear sky,39.18,-84.53,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Use base URL to search for hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,80.60,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
8,Tasgaon,IN,81.57,broken clouds,17.03,74.60,Rajan Patil Mala
9,Vaini,TO,77.00,few clouds,-21.20,-175.20,Keleti Beach Resort
10,Saint-Philippe,RE,75.20,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
24,Ixtapa,MX,82.40,overcast clouds,20.70,-105.20,Container Inn Puerto Vallarta
25,Kapaa,US,80.60,scattered clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
29,Adrar,MR,88.70,clear sky,20.50,-10.07,
41,Kirakira,SB,83.62,light rain,-10.45,161.92,SanBiz Lodge
46,Sao Filipe,CV,77.32,scattered clouds,14.90,-24.50,Tortuga B&B
56,Hamilton,US,78.01,clear sky,39.18,-84.53,Six Acres Bed & Breakfast


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hilo,US,80.60,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
8,Tasgaon,IN,81.57,broken clouds,17.03,74.60,Rajan Patil Mala
9,Vaini,TO,77.00,few clouds,-21.20,-175.20,Keleti Beach Resort
10,Saint-Philippe,RE,75.20,scattered clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
24,Ixtapa,MX,82.40,overcast clouds,20.70,-105.20,Container Inn Puerto Vallarta


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))